# Compare the restricted SCvH EOS tables for a H-He mixture
We compare two EOS tables that are derived from the SCvH equation of state [Saumon et al. 1995](https://ui.adsabs.harvard.edu/abs/1995ApJS...99..713S/abstract) for Hydrogen and Helium. The first is limited to $-8 \leq \log{\rho} \leq -4$ and $1.06 \leq \log{T} \leq 3.46$ and extended to lower densities using (linear) extrapolation. The second covers a much larger range in density ($-14 \leq \log{\rho} \leq -4$) and temperatures ($1.06 \leq \log{T} \leq 3.49$).

### Import modules

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import interpolate as interp
import numpy as np

In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'

In [ ]:
mpl.rcParams['figure.dpi'] = 100

In [ ]:
def read_eos_table_dt(filename, delimiter=None, skiprows=0):
    data = np.loadtxt(filename, delimiter=delimiter, skiprows=skiprows) 

    logT_table   = data[:, 0]
    logrho_table = data[:, 1]
    logP_table   = data[:, 2]
    logu_table   = data[:, 3]
    logs_table   = data[:, 4]

    # All the SCvH EOS tables are tabulated along isotherms
    logT = np.unique(logT_table)
    nT = np.size(logT)

    # For the mixture tables the number of grid points in rho the same for each isotherm
    nRho = int(np.size(logT_table)/nT)
    
    print("nT = {:} nRho = {:}".format(nT, nRho))
    
    logrho = logrho_table[0:nRho]
    
    logrho_min = np.min(logrho)
    logrho_max = np.max(logrho)
    logT_min   = np.min(logT)
    logT_max   = np.max(logT)

    print("logrho_min = {:}".format(logrho_min))
    print("logrho_max = {:}".format(logrho_max))
    print("logT_min   = {:}".format(logT_min))
    print("logT_max   = {:}".format(logT_max))
    print()

    # Split into arrays of constant T
    logP_array = np.split(logP_table, nT)
    logu_array = np.split(logu_table, nT)
    logs_array = np.split(logs_table, nT)

    # Generate 2d arrays
    logP = np.vstack(logP_array)
    logu = np.vstack(logu_array)
    logs = np.vstack(logs_array)
      
    eos_table_dt = {
        "nT":         nT,
        "nRho":       nRho,
        "logT":       logT,
        "logrho":     logrho,
        "logrho_min": logrho_min,
        "logrho_max": logrho_max,
        "logT_min":   logT_min,
        "logT_max":   logT_max,
        "logP":       logP,
        "logu":       logu,
        "logs":       logs,
    }
    
    return eos_table_dt

In [ ]:
def press_ideal_gas(rho, T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/(mu*m_H)*rho*T

In [ ]:
def intenergy_ideal_gas(T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/((gamma-1.0)*mu*m_H)*T

In [ ]:
def entropy_ideal_gas(T, mu=2.3, gamma=5.0/3.0):
    kB = 1.38e-16  # erg/K
    m_H = 1.67e-24 # g
    return kB/(mu*m_H)*(np.log(T/T0) - 1.0/(gamma-1.0)*np.log(rho/rho0))

The mixture was obtained from the additive volume law (AVL) with Y=0.278 using bilinear spline interpolation and extrapolation.

In [ ]:
#scvh_hhe = read_eos_table_dt("scvh_hhe_y0.275_dt_cgs.csv", delimiter=",", skiprows=1)
scvh_hhe_ext = read_eos_table_dt("scvh_extended_dt_hydrogen_722_helium_278.data")
scvh_hhe_extrap = read_eos_table_dt("extrap_scvh_on_grid.txt")

In [ ]:
# Generate interpolation functions for the extended EOS table
logP_int = interp.interp2d(scvh_hhe_ext['logrho'], scvh_hhe_ext['logT'], scvh_hhe_ext['logP'], kind='linear')
logu_int = interp.interp2d(scvh_hhe_ext['logrho'], scvh_hhe_ext['logT'], scvh_hhe_ext['logu'], kind='linear')
logs_int = interp.interp2d(scvh_hhe_ext['logrho'], scvh_hhe_ext['logT'], scvh_hhe_ext['logs'], kind='linear')

In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 4
nSkipRho = 10

#nSkipT = 1
#nSkipRho = 1

In [ ]:
# Plot log(P), log(u) and log(s) of the two tables
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)
ax[0][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nT'], nSkipT):
    ax[0][0].plot(scvh_hhe_extrap['logrho'], scvh_hhe_extrap['logP'][i,:], '-')

ax[0][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nT'], nSkipT):
    ax[0][0].plot(scvh_hhe_extrap['logrho'], logP_int(scvh_hhe_extrap['logrho'], scvh_hhe_extrap['logT'][i]), '--')

# Mark T=T_min
ax[0][0].plot(scvh_hhe_extrap['logrho'], scvh_hhe_extrap['logP'][0,:], '--', color='black')

ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
ax[0][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nRho'], nSkipRho):
    ax[0][1].plot(scvh_hhe_extrap['logT'], scvh_hhe_extrap['logP'][:,i], '-')

ax[0][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nRho'], nSkipRho):
    ax[0][1].plot(scvh_hhe_extrap['logT'], logP_int(scvh_hhe_extrap['logrho'][i], scvh_hhe_extrap['logT']), '--')

# Mark rho=rho_max
index_rho_max = scvh_hhe_extrap['nRho']-1
ax[0][1].plot(scvh_hhe_extrap['logT'], scvh_hhe_extrap['logP'][:,index_rho_max], '--', color='black')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
ax[1][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nT'], nSkipT):
    ax[1][0].plot(scvh_hhe_extrap['logrho'], scvh_hhe_extrap['logu'][i,:], '-')

ax[1][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nT'], nSkipT):
    ax[1][0].plot(scvh_hhe_extrap['logrho'], logu_int(scvh_hhe_extrap['logrho'], scvh_hhe_extrap['logT'][i]), '--')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
ax[1][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nRho'], nSkipRho):
    ax[1][1].plot(scvh_hhe_extrap['logT'], scvh_hhe_extrap['logu'][:,i], '-')

ax[1][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nRho'], nSkipRho):
    ax[1][1].plot(scvh_hhe_extrap['logT'], logu_int(scvh_hhe_extrap['logrho'][i], scvh_hhe_extrap['logT']), '--')

index_rho_max = scvh_hhe_extrap['nRho']-1
ax[1][1].plot(scvh_hhe_extrap['logT'], scvh_hhe_extrap['logu'][:,index_rho_max], '--', color='black')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
ax[2][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nT'], nSkipT):
    ax[2][0].plot(scvh_hhe_extrap['logrho'], scvh_hhe_extrap['logs'][i,:], '-')

ax[2][0].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nT'], nSkipT):
    ax[2][0].plot(scvh_hhe_extrap['logrho'], logs_int(scvh_hhe_extrap['logrho'], scvh_hhe_extrap['logT'][i]), '--')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
ax[2][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nRho'], nSkipRho):
    ax[2][1].plot(scvh_hhe_extrap['logT'], scvh_hhe_extrap['logs'][:,i], '-')

ax[2][1].set_prop_cycle(None)
for i in range(0, scvh_hhe_extrap['nRho'], nSkipRho):
    ax[2][1].plot(scvh_hhe_extrap['logT'], logs_int(scvh_hhe_extrap['logrho'][i], scvh_hhe_extrap['logT']), '--')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")
